How it works: 

This simulation lets you creates an ISA portfolio of 100 to 2000 students right before graduation and tracks monthly payments over time. You are able to input the variables seen below to structure the terms of the ISA program according to a targeted ROI. 

How are payments decided:

Students can occupy 6 different status:
    1. Not Graduated
    2. Grace Period
    3. In Repayment
    4. In Deferment
    5. In Delinquency
    6. End (Term Conditions reached)
    
Using external data (mainly from the census bureau) to initialize income distributions and rates of delinquency, this simulation will transition students between statuses every month according to their current income, income share (higher income shares will be more likely to be delinquent) as well as their specific ISA term conditions.

In [24]:
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import ipywidgets as widgets
from IPython.display import display, clear_output

# counter variables
num_sim = 1000
time_count = 0
repayment_per_month = []
monthly_payarray = []
status_count = []

# independent variables
num_stu = 0
pay_cap = 0
pay_terms = 0
exp_terms = 0
exp_multiple = 0
min_dis = 0
max_dis = 0
min_income = 0
income_growth_rate = 0
grad_rate_4year = 0
total_disbursement = 0

# widget layout
layout = widgets.Layout(width='400px', height='40px')

# widget list

output_init = widgets.Output()
output_runsim = widgets.Output()

numstu_widget = widgets.IntSlider(min=100, max=2000, step=100, description='Students: ', value=1000,
                                  readout_format='d', layout=layout)

paycap_widget = widgets.FloatSlider(min=1.0, max=2.5, step=0.25, description='PayCap: ', value=2.0,
                                    readout_format='.0%', layout=layout)

payterms_widget = widgets.IntSlider(min=2, max=12, step=1, description='TermsYrs: ', value=5, layout=layout)

expterms_widget = widgets.IntSlider(min=5, max=20, step=1, description='ExpTermsYrs', value=10, layout=layout)

mindis_widget = widgets.FloatSlider(min=1000, max=5000, step=1000, description='MinDisb: ', value=2000,
                                 readout_format='$', layout=layout)

maxdis_widget = widgets.FloatSlider(min=5000, max=25000, step=1000, description='MaxDisb: ', value=20000,
                                 readout_format='$', layout=layout)

mincome_widget = widgets.IntSlider(min=20000, max=60000, step=5000, description='MinIncome: ', value=30000,
                                   readout_format = '$', layout=layout)

expm_widget = widgets.FloatSlider(min=1.0, max=2.5, step=0.1, description='TargetROI: ', value=1.5,
                                  readout_format='.0%', layout=layout)

igr_widget = widgets.FloatSlider(min=0.01, max=0.1, step=0.01, description='IncGrowthR: ', value=0.05,
                                 readout_format='.0%', layout=layout)

grad_widget = widgets.FloatSlider(min=0.6, max=1.0, step=0.05, description='4YrGrad: ', value=0.7,
                                  readout_format='.0%', layout=layout)

init_widget = widgets.Button(description='Initialize Variables', disabled=False,
                               button_style='success', icon='cog')

runsim_widget = widgets.Button(description='Run Simulation', disabled=False,
                               button_style='success', icon='play')


In [25]:
#functions

def starting_income(num_students):
    my_arr = []
    for i in range(num_students):
        inc_below_25 = np.random.choice(np.arange(28000, 40000, 1000))
        inc_25_to_75 = np.random.choice(np.arange(41000, 68000, 1000))
        inc_75_to_95 = np.random.choice(np.arange(69000, 85000, 1000))
        inc_95 = np.random.choice(np.arange(86000, 90000, 1000))
        my_arr.append(np.random.choice([inc_below_25, inc_25_to_75, inc_75_to_95, inc_95], p=[0.25, 0.5, 0.20, 0.05]))
    return my_arr

def income_growth(curr_inc, growth_rate):
    return curr_inc*(1+growth_rate)

def income_share(dis_amt, start_inc, pay_terms, exp_payout_multiple):
    inc_arr = []
    years = int(pay_terms/12)
    inc = start_inc
    for i in range(years):
        inc_arr.append(inc)
        inc = income_growth(inc, income_growth_rate)
        
    total_inc = np.sum(inc_arr)    
    mean_yr_inc = total_inc/years
    exp_payout = dis_amt*exp_payout_multiple
    
    yearly_pay = exp_payout/years
    income_sh = yearly_pay/mean_yr_inc
    return income_sh

def monthly_pay(curr_inc, inc_share):
    if curr_inc >= min_income:
        return (curr_inc*inc_share)/12
    else:
        return 0
    
def income_initialize(df):
    for i, row in df.iterrows():
        curr_inc_val = 0
        if row['Status'] == 1:
            curr_inc_val = row['Expected Starting Income']
            df.at[i,'Current Income'] = curr_inc_val
    return df

def yearly_update(time_count, df):
    if time_count == 24: # after 2 years, if student hasn't graduated, they are assumed to have dropped out
        for i, row in df.iterrows():
            if row['Status'] == 0:
                df.at[i, 'Status'] = 5
    if time_count == 12: # after 1 year, x% chance that a student who hasn't graduated will graduate
        for i, row in df.iterrows():
            if row['Status'] == 0:
                    df.at[i, 'Status'] = np.random.choice([0, 1], p=[0.5, 0.5])
    elif(time_count%12) == 0:
        for i, row in df.iterrows():
            df.at[i, 'Current Income'] = income_growth(row['Current Income'], income_growth_rate)
                
def grace_update(time_count, df):
    if (time_count%6) == 0:
        for i, row in df.iterrows():
            if row['Status'] == 1:
                df.at[i, 'Status'] = 2
                df.at[i,'Current Income'] = row['Expected Starting Income']
                
                
def status_update(df):
    for i, row in df.iterrows():
        if row['Status'] == 5:
            pass
        else:
            mp = round(monthly_pay(row['Current Income'], row['Income Share']))

            # income doesn't meet threshold, go to deferment

            if mp == 0:
                if row['Status'] == 0:
                    df.at[i, 'Status'] = 0
                elif row['Status'] == 2:
                    df.at[i, 'Status'] = 3
                elif row['Status'] == 3:
                    df.at[i, 'Status'] = 3
                elif row['Status'] == 4:
                    df.at[i, 'Status'] = 3

            if mp != 0 and row['Status'] == 3:
                df.at[i, 'Status'] = 2

            # meets max owed cap
            if row['Pay Terms'] < 1:
                df.at[i, 'Status'] = 5
            elif row['Max Owed'] < (row['Paid Off'] + mp):
                df.at[i, 'Paid Off'] = row['Max Owed']
                df.at[i, 'Status'] = 5


            # random 2% chance to go into delinquency every month
            if row['Status'] == 2:
                delinquent = np.random.choice([0, 1], p=[0.05, 0.95])
                if delinquent == 0:
                    df.at[i, 'Status'] = 4
                else:
                    df.at[i, 'Paid Off'] += mp
                    df.at[i, 'Pay Terms'] -= 1 

            # random 50% chance of delinquents to go back into repayment
            elif row['Status'] == 4:
                non_delinquent = np.random.choice([0, 1], p=[0.5, 0.5])
                if non_delinquent == 1:
                    df.at[i, 'Status'] = 2
                    df.at[i, 'Paid Off'] += mp
                    df.at[i, 'Pay Terms'] -= 1
                
    return df



In [26]:
# status map: 0=student, 1=graceperiod, 2=repayment, 3=deferment, 4=delinquent, 5=end
# simulation functions

# samples creation
def samples_create():
    global total_disbursement
    status = np.zeros(num_stu)
    income = np.zeros(num_stu)
    expected_income = starting_income(num_stu)
    dis_amt = np.random.choice(np.arange(min_dis, max_dis, 500), num_stu)
    pay_terms_arr = np.full(num_stu, pay_terms, dtype=int)
    income_share_list = []
    for i in range(len(expected_income)):
        income_share_list.append(income_share(dis_amt[i], expected_income[i], pay_terms_arr[i], exp_multiple))

    max_owed = dis_amt*pay_cap
    paid_off = np.zeros(num_stu)
    students = np.stack((status, income, expected_income, dis_amt, 
                         max_owed, income_share_list, pay_terms_arr, paid_off), axis=-1)

    df = pd.DataFrame(data=students, columns=['Status', 'Current Income', 'Expected Starting Income',
                                              'Disbursement Amount', 'Max Owed', 'Income Share', 
                                              'Pay Terms', 'Paid Off'])

    df['Income Share'] = pd.Series([round(val, 4) for val in df['Income Share']], index = df.index)

    total_disbursement = df['Disbursement Amount'].sum()
    print('\nTotal Disbursement Amount: ', total_disbursement)
    print("Months Passed :", time_count)
    return df


# initial graduation update
def graduate(df):
    df['Status'] = np.transpose(np.random.choice([0, 1], num_stu, p=[1-grad_rate_4year, grad_rate_4year]))
    #sns.countplot(df['Status'])
    df = income_initialize(df)
    return df

# after grace period
def grace_period(df):
    global time_count
    for i in range(6):
        time_count += 1
        repayment_per_month.append(df['Paid Off'].sum())

    for i, row in df.iterrows():
        if row['Status'] == 1:
            inc_owed = monthly_pay(row['Current Income'], row['Income Share'])
            if inc_owed == 0:
                df.at[i, 'Status'] = 3
            else:
                df.at[i, 'Status'] = 2

    print("Months Passed : ", time_count)
    print("Repayments Array : ", repayment_per_month)
    #sns.countplot(df['Status'])
    return df


# simulate
def simulate(df):
    global repayment_per_month
    global time_count
    global status_count
    for i in range(exp_terms):
        yearly_array = []
        for j in range(12):
            time_count += 1
            grace_update(time_count, df)
            yearly_update(time_count, df)
            df = status_update(df)


            #for i, row in df.iterrows():
            #   inc_owed = round(monthly_pay(row['Current Income'], row['Income Share']))
            #   if row['Status'] == 1:
            #       if inc_owed == 0:
            #           df.at[i, 'Status'] = 3
            #       else:
            #           df.at[i, 'Status'] = 2
            #   if row['Status'] == 2:
            #       df.at[i, 'Paid Off'] += inc_owed

            repayment_per_month.append(round(df['Paid Off'].sum()))
            
        for k in range(time_count-13, time_count-1):
            temp = repayment_per_month[k+1] - repayment_per_month[k]
            yearly_array.append(temp)
        print("\nMonths Passed: ", time_count)
        print("Repayments Array: ", yearly_array)
        status_count.append(df['Status'].value_counts())

    #sns.countplot(df['Status'])
    #print('\n', status_count)
    return df

In [27]:
# plotting functions

def plot_returns():
    roi = (repayment_per_month/total_disbursement)
    roi -= 1
    plt.plot(np.arange(time_count), roi)
    plt.xlabel('Months')
    plt.ylabel('Total Return on Investment')
    plt.show()
    
def plot_monthly_payments():
    global monthly_payarray
    for i in range(len(repayment_per_month)-1):
        temp = repayment_per_month[i+1] - repayment_per_month[i]
        monthly_payarray.append(temp)

    plt.plot(np.arange(time_count-1), monthly_payarray)
    plt.xlabel('Months')
    plt.ylabel('Dollars paid back')
    plt.show()
    
def plot_yearly_payments():
    yield_arr = monthly_payarray/total_disbursement
    year_yield = np.add.reduceat(yield_arr, np.arange(0, len(yield_arr), 12))
    plt.plot(np.arange(1, (time_count/12)+1), year_yield)
    plt.xlabel('Year')
    plt.ylabel('Coupon Rate')
    plt.show()
    print('Coupon Rate per year: ',[f'{i*100:.1f}%' for i in year_yield])

In [28]:
# widget functions
def initfunc(b=None):
    global num_stu
    global pay_cap
    global pay_terms
    global exp_terms
    global exp_multiple
    global min_income
    global min_dis
    global max_dis
    global income_growth_rate
    global grad_rate_4year
    num_stu = numstu_widget.value
    pay_cap = paycap_widget.value
    pay_terms = payterms_widget.value*12
    exp_terms = expterms_widget.value
    min_dis = mindis_widget.value
    max_dis = maxdis_widget.value
    exp_multiple = expm_widget.value
    min_income = mincome_widget.value
    income_growth_rate = igr_widget.value
    grad_rate_4year = grad_widget.value
    with output_init:
        print('\nINPUTS SELECTED')
        print('\nNumber of Students: ', num_stu)
        print('\nDisbursement Range: ', '${:,.0f}'.format(min_dis),'-', '{:,.0f}'.format(max_dis))
        print('\nPayment Cap: ', '{:.0%}'.format(pay_cap))
        print('\nNumber of Pay Terms: ', pay_terms/12, 'Years')
        print('\nExpiration Length: ', exp_terms, 'Years')
        print('\nTarget ROI: ', '{:.2%}'.format(exp_multiple))
        print('\nMinimum Income Threshold: ', '${:,.2f}'.format(min_income))
        print('\nIncome Growth Rate: ', '{:.2%}'.format(income_growth_rate))
        print('\n4-Year Graduation Rate: ', '{:.0%}'.format(grad_rate_4year))
        display(runsim_widget)


def runsim(b=None):
    with output_runsim:
        sim_df = samples_create()
        print('\n\nInitializing DataFrame of %d Students' %num_stu)
        display(sim_df.head(10))
        sim_df = graduate(sim_df)
        sim_df = grace_period(sim_df)
        print('\n\nStudent DataFrame with Income Initialization')
        display(sim_df.head(10))
        sim_df = simulate(sim_df)
        print('\n\nFinal Dataframe')
        display(sim_df.head(10))
        print('Total Returns: ', repayment_per_month[-1])
        print('Return on Investment: ', '{:.2%}'.format((repayment_per_month[-1]/total_disbursement)))
        print('\n Monthly Payments Graph')
        plot_monthly_payments()
        print('\n Yearly Payments Graph')
        plot_yearly_payments()

In [29]:
# display widgets

display(numstu_widget, 
        paycap_widget, 
        payterms_widget,
        expterms_widget,
        mincome_widget,
        mindis_widget,
        maxdis_widget,
        expm_widget, 
        igr_widget, 
        grad_widget,
        init_widget) 

init_widget.on_click(initfunc)
display(output_init)
runsim_widget.on_click(runsim)
display(output_runsim)

IntSlider(value=1000, description='Students: ', layout=Layout(height='40px', width='400px'), max=2000, min=100…

FloatSlider(value=2.0, description='PayCap: ', layout=Layout(height='40px', width='400px'), max=2.5, min=1.0, …

IntSlider(value=5, description='TermsYrs: ', layout=Layout(height='40px', width='400px'), max=12, min=2)

IntSlider(value=10, description='ExpTermsYrs', layout=Layout(height='40px', width='400px'), max=20, min=5)

IntSlider(value=30000, description='MinIncome: ', layout=Layout(height='40px', width='400px'), max=60000, min=…

...

FloatSlider(value=0.05, description='IncGrowthR: ', layout=Layout(height='40px', width='400px'), max=0.1, min=…

FloatSlider(value=0.7, description='4YrGrad: ', layout=Layout(height='40px', width='400px'), max=1.0, min=0.6,…

Button(button_style='success', description='Initialize Variables', icon='cog', style=ButtonStyle())

Output()

Output()